In [53]:
import requests
import streamlit as st
import pandas as pd
#import harp
from netCDF4 import Dataset
import os
from os.path import join
import glob
import xarray as xr
import numpy as np
from numpy import array
import itertools
import datetime
import time
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt 
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import base64

In [70]:
L3_data1=Dataset(r"5days_combined.nc")
print(L3_data1.variables.keys())
lat=L3_data1.variables['latitude'][:]
lon=L3_data1.variables['longitude'][:]
time_data=L3_data1.variables['time'][:]
no2=L3_data1.variables['tropospheric_NO2_column_number_density'][0:,:,:]
print("lat", lat)
print("lon", lon)
print("no2", no2)
print("time",time_data )

dict_keys(['tropospheric_NO2_column_number_density', 'latitude', 'longitude', 'time'])
lat [ 8.005  8.015  8.025 ... 36.965 36.975 36.985]
lon [68.005 68.015 68.025 ... 96.965 96.975 96.985]
no2 [[[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... 0.8934862801990139 0.8934862801997281 0.9101298225528585]
  [-- -- -- ... 0.893486280199371 0.8934862801990933 1.0535357590655705]
  [-- -- -- ... 0.8934862801990139 0.9001095148394141 1.2166691595353685]]

 [[-- -- -- ... 0.39692594058520886 0.17109398945177076
   0.17109398945196072]
  [-- -- -- ... 0.3062241037452048 0.17109398945177076
   0.17109398945196072]
  [-- -- -- ... 0.2151974610707862 0.1710944610101049
   0.17109398945196072]
  ...
  [1.326423744628536 1.3264237446289484 1.326423744630009 ...
   0.774007264747734 0.7740072647483527 0.7740072647474933]
  [1.3264237446290663 1.3264237446294787 1.3264237446290663 ...
   0.7740072647480433 0.7740072647478027 0.7740072647478027]
  [1.326

In [73]:
def predict(latitude_input,longitude_input,date):
    

   
    predict_days=abs((datetime.strptime('2024-09-05',"%Y-%m-%d")-datetime.strptime(date,"%Y-%m-%d")).days)
    st.write("--------predict days-----:",predict_days)
    
    
    #
    sq_diff_lat=(lat-latitude_input)**2
    sq_diff_lon=(lon-longitude_input)**2
    
    min_index_lat=sq_diff_lat.argmin()
    min_index_lon=sq_diff_lon.argmin()
    
    start_date=L3_data1.variables['time'].units[14:24]
    end_date=L3_data1.variables['time'].units[14:18]+'-09-05'
    
    date_range=pd.date_range(start=start_date,end=end_date)
    st.success(date_range)   
    df=pd.DataFrame(0,columns=['NO2'],index=date_range)
    print(df)
    dt=np.arange(0,5)

    print(dt)
    
    for i in dt:
        df.iloc[i]=no2[i,min_index_lat,min_index_lon]
        
    df.to_csv(r"5days_combined.csv")
    #print(os.path.getsize(r"C:\Users\HP\.spyder-py3\data\days_combined.csv"))
    
    ##############################################
    #            PREDICTION MODULE               #
    ##############################################
    ### Data Collection
    data_frame=pd.read_csv(r"5days_combined.csv")
    df1=data_frame.reset_index()['NO2']
    df1 = df1.fillna(0)
    print(df1)
    ### LSTM are sensitive to the scale of the data. so we apply MinMax scaler 

    scaler=MinMaxScaler(feature_range=(0,1))
    df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
    
    ##splitting dataset into train and test split


    training_size=int(len(df1)*0.90)
    test_size=len(df1)-training_size
    train_data,test_data=df1[0:training_size,:],df1[test_size:len(df1),:]
    print(train_data)
    print(test_data)
       
    
    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):         
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)
    
    # reshape into X=t,t+1,t+2,t+3 and Y=t+4
    time_step = 1
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, ytest = create_dataset(test_data, time_step)

    print(X_train.shape), print(y_train.shape)
    print(X_test.shape), print(ytest.shape)

    
    # print(X_test.shape), print(ytest.shape)
  
    # reshape input to be [samples, time steps, features] which is required for LSTM
    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
    print(X_train)
    
    ### Create the Stacked LSTM model
    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(1,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    
    
    model.summary()
    model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=25,batch_size=2,verbose=1)
    

    ### Lets Do the prediction and check performance metrics
    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)
    
    ##Transformback to original form
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)
    
    ### Calculate RMSE performance metrics
    
    math.sqrt(mean_squared_error(y_train,train_predict))
    
    ### Test Data RMSE
    math.sqrt(mean_squared_error(ytest,test_predict))
    
    ### Plotting 
    # shift train predictions for plotting
    look_back=1
    trainPredictPlot = np.empty_like(df1)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(df1)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict
      
    x_input=test_data[len(test_data)-1:].reshape(1,-1)
    
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()
    
    # demonstrate prediction for next days
    
    lst_output=[]
    n_steps=1
    i=0
    while(i<predict_days):
        
        if(len(temp_input)>1):
            #print(temp_input)
            x_input=np.array(temp_input[1:])
            print("{} day input {}".format(i,x_input))
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
            print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            print(yhat[0])
            temp_input.extend(yhat[0].tolist())
            print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i=i+1
        
    
    print(lst_output)
        
    # st.write(df3)
    no2_output=pd.DataFrame(scaler.inverse_transform(lst_output),columns=['NO2 Concentration ðŸ­'])
    print(no2_output)
    output= (no2_output.at[predict_days-1,'NO2 Concentration ðŸ­'])
    return output

In [74]:

latitude_input=19.3
longitude_input=19.3
date= "2024-09-09"
result = predict(latitude_input,longitude_input,date)

2024-09-29 02:36:46.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 02:36:46.255 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 02:36:46.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 02:36:46.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 02:36:46.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 02:36:46.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\PRAMILA\AppData\Roaming\Python\Python39\site-packages\pandas\core\internals\blocks.py:1067: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted


            NO2
2024-09-01    0
2024-09-02    0
2024-09-03    0
2024-09-04    0
2024-09-05    0
[0 1 2 3 4]
0    0.000000
1    0.000000
2    0.255614
3    0.000000
4    0.342948
Name: NO2, dtype: float64
[[0.       ]
 [0.       ]
 [0.7453447]
 [0.       ]]
[[0.       ]
 [0.7453447]
 [0.       ]
 [1.       ]]
(2, 1)
(2,)
(2, 1)
(2,)
[[[0.]]

 [[0.]]]
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_39 (LSTM)              (None, 1, 50)             10400     
                                                                 
 lstm_40 (LSTM)              (None, 1, 50)             20200     
                                                                 
 lstm_41 (LSTM)              (None, 50)                20200     
                                                                 
 dense_13 (Dense)            (None, 1)                 51        
                               

ValueError: could not broadcast input array from shape (2,1) into shape (0,1)